### Install bs4 package that contains BeautifulSoup

In [1]:
#!pip install bs4

### Load all required packages to save the Wikipedia HTML page

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
bs_src = BeautifulSoup(html.read(),'lxml');

### Extract table and form a pandas dataframe with headers

In [3]:
import pandas as pd
import numpy as np

xml_table = bs_src.find('table',{'class':'wikitable sortable'})

headers=xml_table.find_all('th')
headers=[x.text.strip() for x in headers]
table=pd.DataFrame([])

rows = xml_table.find_all('tr')
for row in rows:
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols]
    table=pd.concat([table,pd.DataFrame([cols])],axis=0)

table.columns=headers
table.head()

,Postcode,Borough,Neighbourhood
0,NaN,NaN,NaN
0,M1A,Not assigned,Not assigned
0,M2A,Not assigned,Not assigned
0,M3A,North York,Parkwoods
0,M4A,North York,Victoria Village


### Preprocess table

In [4]:
table=table.dropna(axis=0)
table=table[table.Borough!='Not assigned']
table['Postcode'] = table['Postcode'].astype(str)
table['Borough'] = table['Borough'].astype(str)
table['Neighbourhood'] = table['Neighbourhood'].astype(str)
for j in range(len(table)):
    if table.Neighbourhood.iloc[j] == 'Not assigned':
        table.Neighbourhood.iloc[j]=table.Borough.iloc[j]
table.reset_index(drop=True, inplace=True)
table = table.groupby(['Postcode','Borough'],as_index=False).agg( ', '.join)
table.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Print table dimensions

In [5]:
table.shape

(103, 3)